In [ ]:
%pip install sagemaker boto3

In [1]:
import json
import sagemaker
import boto3
import os
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri


In [2]:
#print(os.environ['AWS_PROFILE'])
os.environ['AWS_PROFILE'] = 'isengardmain'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'


## Tests

In [3]:
try:
    #session = boto3.Session(profile_name='isengardmain')
    # print(session)
    # c = session.client("sts")
    # print(c.get_caller_identity())
    #sagemaker_session = sagemaker.Session(boto_session=session)
    #role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)
    #role = sagemaker.get_execution_role()
    #role = iam.get_role(RoleName='Admin')['Role']['Arn']
    #print(role)
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20221220T115946', )['Role']['Arn']
except ValueError:
    print("default:")
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role', )['Role']['Arn']
print(role)

arn:aws:iam::433808754371:role/service-role/AmazonSageMaker-ExecutionRole-20221220T115946


In [4]:
# Hub Model configuration. https://huggingface.co/models
hub = {
 'HF_MODEL_ID':'tiiuae/falcon-40b-instruct',
 'SM_NUM_GPUS': json.dumps(4)
}

In [6]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
    env=hub,
    role=role, 
)

In [7]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.12xlarge",
    container_startup_health_check_timeout=300,
)

----------------!

In [11]:
# send request
predictor.predict({
 "inputs": "My name is Flolight and I like to",
})

[{'generated_text': "My name is Flolight and I like to play with my friends.\nI'm sorry, but I am an AI language model and I"}]